# BERTopic-based Subcategory Extraction

This notebook discovers subcategories per main category using BERTopic with strong sentence embeddings and tuned clustering. It also standardizes labels and provides quick evaluations.


In [87]:
# Imports
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from sklearn.feature_extraction.text import TfidfVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

import torch
from tqdm.auto import tqdm
tqdm.pandas()


In [88]:
# Load data
DATA_PATH = "../data/raw_bbc.csv"
df = pd.read_csv(DATA_PATH)
assert {"Category", "Text"}.issubset(df.columns), "CSV must have Category and Text columns"

# Keep short titles for embedding and topic representation
def make_title(text, max_words=40):
    words = str(text).split()
    return " ".join(words[:max_words])

df["Title"] = df["Text"].apply(make_title)
df.head(3)


,Category,Text,Filename,Subcategory,Title
0,business,"Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.\n\nThe firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.\n\nTime Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.\n\nTime Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. ""Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,"" chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.\n\nTimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.\n",data/business/001.txt,NaN,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited"
1,business,"Dollar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\n\nAnd Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. ""I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time,"" said Robert Sinche, head of currency strategy at Bank of America in New York. ""He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next.""\n\nWorries about th

In [89]:
# Clean text for BERTopic input
import re
import nltk
from nltk.corpus import stopwords

try:
    stop_words = set(stopwords.words("english"))
except LookupError:
    nltk.download("stopwords")
    stop_words = set(stopwords.words("english"))

WORD_RE = re.compile(r"[A-Za-z]+")

def clean_text(text: str, max_words: int = 200) -> str:
    tokens = str(text).lower().split()
    tokens = tokens[:max_words]
    cleaned = []
    for tok in tokens:
        if not WORD_RE.fullmatch(tok):
            continue
        if tok in stop_words:
            continue
        if len(tok) <= 2:
            continue
        cleaned.append(tok)
    return " ".join(cleaned)

# Build cleaned content from full Text
# Use first 200 tokens to balance quality/speed and avoid seq length issues
df["CleanContent"] = df["Text"].apply(lambda x: clean_text(x, max_words=200))
df.head(3)


,Category,Text,Filename,Subcategory,Title,CleanContent
0,business,"Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.\n\nThe firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.\n\nTime Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.\n\nTime Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. ""Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,"" chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.\n\nTimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.\n",data/business/001.txt,NaN,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited",sales boost time warner profit quarterly profits media giant timewarner jumped three months one biggest investors benefited sales internet connections higher advert timewarner said fourth quarter sales rose profits buoyed gains offset profit dip warner less users time warner said friday owns internet mixed lost subscribers fourth quarter profits lower preceding three company said underlying profit exceptional items rose back stronger internet advertising hopes increase subscribers offering online service free timewarner internet customers try sign existing customers timewarner also restate results following probe securities exchange commission close time fourth quarter profits slightly better
1,business,"Dollar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\n\nAnd Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, th

In [93]:
df.shape

(2225, 6)

In [94]:
# to see the whole dataframe
pd.set_option('display.max_colwidth', None)


In [97]:
# Filter only "business" category and select required columns
filtered = df[df["Category"] == "politics"][["Category", "CleanContent"]]

# Show the result
print(filtered.head(3))




     Category  \
896  politics   
897  politics   
898  politics   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  CleanContent  
896                                                                                                                                                           labour plans materni

In [ ]:
print()

0       False
1       False
2       False
3       False
4       False
        ...  
2220    False
2221    False
2222    False
2223    False
2224    False
Name: is_sports, Length: 2225, dtype: bool


In [ ]:
# Embedding model selection and device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "intfloat/multilingual-e5-large"

embedder = SentenceTransformer(MODEL_NAME, device=DEVICE)
print("Device:", DEVICE)


Device: cuda


In [ ]:
# Encode cleaned content in batches
BATCH_SIZE = 64
corpus = df["CleanContent"].tolist()
embeddings = embedder.encode(corpus, batch_size=BATCH_SIZE, show_progress_bar=True, normalize_embeddings=True)
len(embeddings)


Batches: 100%|██████████| 35/35 [00:44<00:00,  1.27s/it]


2225

In [ ]:
# Category-specific stopwords and bigram vectorizer
SPORTS_STOPWORDS = {
    "england","united","chelsea","arsenal","liverpool","manchester","wenger","ferguson","rooney","premier","cup","league"
}
POLITICS_STOPWORDS = {
    "blair","brown","howard","labour","labour's","tory","tories","conservative","conservatives","minister","government","tony","gordon"
}

# Global baseline stopwords to reduce country/person dominance across all
GLOBAL_EXTRA = {"england","scotland","wales","britain","uk","bbc"}

custom_stopwords = SPORTS_STOPWORDS | POLITICS_STOPWORDS | GLOBAL_EXTRA

vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    lowercase=True,
    stop_words=list(custom_stopwords),
    min_df=.2,
    max_df=.8)


In [ ]:
# Update representations using custom vectorizer + KeyBERT-inspired
# This will recompute topic words without refitting the clustering

try:
    topic_model.set_representations(representation_model=keybert_repr, vectorizer_model=vectorizer)
    rep = topic_model.get_topic_info()
    print("Updated topic representations.")
except Exception as e:
    print("Failed to update representations:", e)


Failed to update representations: 'BERTopic' object has no attribute 'set_representations'


In [ ]:
# UMAP and HDBSCAN configuration
umap_model = UMAP(n_neighbors=25, n_components=5, min_dist=0.0, metric="cosine", random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=5, metric="euclidean", cluster_selection_epsilon=0.0, cluster_selection_method="eom")

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    calculate_probabilities=False,
    verbose=True,
    nr_topics=None,
    min_topic_size=20,
    top_n_words=10
)


In [ ]:
# Fit BERTopic globally, then map within each main category if needed
# Global fit tends to find coherent cross-category subtopics; we will post-label per-category

topics, probs = topic_model.fit_transform(docs_for_topics, embeddings)
df["Topic"] = topics

# Replace -1 with a placeholder
UNKNOWN_LABEL = "Miscellaneous"
df["Topic"] = df["Topic"].fillna(-1)



2025-09-30 15:50:11,088 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


2025-09-30 15:50:26,212 - BERTopic - Dimensionality - Completed ✓
2025-09-30 15:50:26,213 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-30 15:50:26,315 - BERTopic - Cluster - Completed ✓
2025-09-30 15:50:26,327 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-30 15:50:26,733 - BERTopic - Representation - Completed ✓


In [ ]:
# Generate human-readable labels for topics
# Use BERTopic's built-in topic representations

rep = topic_model.get_topic_info()
rep.head(10)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,155,-1_people_digital_technology_computer,"[people, digital, technology, computer, said, use, new, could, music, one]","[horizon super wireless data networks could soon use wireless watchdog seeking help best way regulate technology behind networks called ultra wideband ofcom wants ensure arrival devices cause problems already use part radio uwb makes possible stream huge amounts data air short one likely uses uwb make possible send dvd quality video images wirelessly screens let people beam music media players around technology potential transmit hundreds megabits data per uwb could also used create personal area networks let gadgets quickly easily swap data amongst technology works range metres uses billions short radio pulses every second carry recent consumer electronics show las vegas products uwb chips got first public use uwb, speak easy plan media players music film fans able control digital media players speaking plans development two scansoft gracenote developing technology give people access film music libraries simply voice want give people access digital music films home huge media libraries players make finding single songs unlocks potential devices store large digital music said ross vice president business development applications radically change car entertainment allowing drivers enjoy entire music collections without ever taking hands steering gracenote provides music library information millions different albums jukeboxes new technology designed people play individual song movie saying users also able request music fits mood film saying, speak easy plan media players music film fans able control digital media players speaking plans development two scansoft gracenote developing technology give people access film music libraries simply voice want give people access digital music films home huge media libraries players make finding single songs unlocks potential devices store large digital music said ross vice president business development applications radically change car entertainment allowing drivers enjoy entire music collections without ever taking hands steering gracenote provides music library information millions different albums jukeboxes new technology designed people play individual song movie saying users also able request music fits mood film saying]"
1,0,474,0_said_growth_oil_sales,"[said, growth, oil, sales, firm, shares, company, market, bank, economy]","[sees world economy behemoth general electric posted jump quarterly declared great benefiting growth initiatives excellent global said chief executive jeff biggest firm based stock market net profits final three months sales came whose businesses range jet engines nbc television forecast sustained growth year shares rose news ending friday industries financial industrial sectors picking said steve analyst fund manager matrix asset shares said orders fourth quarter higher period growth across fourth nine businesses delivered least earnings said full year gains less still net profit, barclays profits hit record level seen annual profits climb record levels boosted sharp rise business investment profits year december rose chief john varley said bank strong world earnings barclays capital investment bank rose investment branch operations held back growth retail group first five big banks report according biggest bank stock market report profits later barclays results line market global investors wing made jump profits barclaycard rose said affected series interest rate rises investment grow customer bank also blamed margins pressure mortgage business spending branches past year fall profits retail division outlook, irish markets reach high irish shares risen record investors persuaded buy market low inflation strong growth iseq index leading shares closed points fuelled strong growth banking financial fall rate inflation january gave fresh boost shares advanced economy set strong growth int

In [ ]:
# Refresh representations and rebuild labels with sports/politics consolidation
# 1) Update topic representations now that the model is fitted
try:
    topic_model.set_representations(representation_model=keybert_repr, vectorizer_model=vectorizer)
    rep = topic_model.get_topic_info()
    print("Updated topic representations with bigrams and custom stopwords.")
except Exception as e:
    print("Failed to update representations:", e)

# 2) Build a label map from top words

def build_label_from_words(words, max_terms=3):
    if not words:
        return UNKNOWN_LABEL
    terms = [w for w, _ in words[:max_terms]] if isinstance(words[0], tuple) else words[:max_terms]
    return ", ".join(terms)

# topic -> label
topic_to_label = {}
for topic_id in rep["Topic"].tolist():
    if topic_id == -1:
        topic_to_label[topic_id] = UNKNOWN_LABEL
        continue
    words = topic_model.get_topic(topic_id) or []
    topic_to_label[topic_id] = build_label_from_words(words, max_terms=3)

# 3) Expanded taxonomy mapping for Sports and Politics
CANON = {
    # Sports variants
    "football": "Football",
    "soccer": "Football",
    "premier": "Football",
    "champions league": "Football",
    "fa cup": "Football",
    "cricket": "Cricket",
    "tennis": "Tennis",
    "rugby": "Rugby",
    "olympic": "Olympics",
    "grand slam": "Tennis",
    # Entertainment
    "film": "Movies",
    "movie": "Movies",
    "cinema": "Movies",
    # Business/Economy
    "rate": "Interest Rates",
    "inflation": "Inflation",
    "gdp": "Economy",
    # Politics
    "election": "Elections",
    "vote": "Elections",
    "parliament": "Parliament",
    "policy": "Public Policy",
    "budget": "Budget",
}

def consolidate(label: str) -> str:
    if not label or label == UNKNOWN_LABEL:
        return UNKNOWN_LABEL
    lower = label.lower()
    for key, canon in CANON.items():
        if key in lower:
            return canon
    return label.title()

# Assign labels
labels = [consolidate(topic_to_label.get(t, UNKNOWN_LABEL)) for t in df["Topic"]]
df["Subcategory"] = labels

df.head(5)


Failed to update representations: 'BERTopic' object has no attribute 'set_representations'


,Category,Text,Filename,Subcategory,Title,CleanContent,Topic
0,business,"Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.\n\nThe firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.\n\nTime Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.\n\nTime Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. ""Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,"" chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.\n\nTimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.\n",data/business/001.txt,"Said, Growth, Oil","Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited",sales boost time warner profit quarterly profits media giant timewarner jumped three months one biggest investors benefited sales internet connections higher advert timewarner said fourth quarter sales rose profits buoyed gains offset profit dip warner less users time warner said friday owns internet mixed lost subscribers fourth quarter profits lower preceding three company said underlying profit exceptional items rose back stronger internet advertising hopes increase subscribers offering online service free timewarner internet customers try sign existing customers timewarner also restate results following probe securities exchange commission close time fourth quarter profits slightly better,0
1,business,"Dollar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\n\nAnd Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late

In [ ]:
# Evaluation: top subcategories per main category (after consolidation)
view = (
    df.groupby(["Category", "Subcategory"])  # type: ignore
      .size()
      .reset_index(name="Count")
      .sort_values(["Category", "Count"], ascending=[True, False])
)
view.head(20)


,Category,Subcategory,Count
7,business,"Said, Growth, Oil",464
8,business,"Said, Would, Labour",23
1,business,"England, Six, Game",7
3,business,Miscellaneous,5
10,business,"Security, Users, Microsoft",3
0,business,"Broadband, Net, Service",2
2,business,"Games, Game, Gaming",2
4,business,"Mobile, Phone, Phones",1
5,business,Movies,1
6,business,"Music, Apple, Industry",1


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def clean_label(label):
    if pd.isna(label) or label == "":
        return "Miscellaneous"
    # split words by commas or underscores
    tokens = [t.strip() for t in label.replace("_", " ").split(",")]
    # filter stopwords and short tokens
    tokens = [t for t in tokens if t.lower() not in stop_words and len(t) > 2]
    # return clean label
    return ", ".join(sorted(set(tokens))) if tokens else "Miscellaneous"

# clean all labels
df["Subcategory"] = df["Subcategory"].apply(clean_label)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cyrus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Save outputs (robust to missing parquet engines)
OUT_CSV = "bbc_with_subcategories_bertopic.csv"
OUT_PARQUET = "bbc_with_subcategories_bertopic.parquet"

cols = ["Category", "Text", "Topic", "Subcategory"]

# Always save CSV
df[cols].to_csv(OUT_CSV, index=False)

# Try Parquet if engine available; otherwise skip with a note
parquet_saved = False
try:
    import importlib
    if importlib.util.find_spec("pyarrow") or importlib.util.find_spec("fastparquet"):
        df[cols].to_parquet(OUT_PARQUET, index=False)
        parquet_saved = True
    else:
        print("Parquet engine not installed (pyarrow/fastparquet). Skipping Parquet save.")
except Exception as e:
    print(f"Parquet save failed: {e}")

(OUT_CSV, OUT_PARQUET if parquet_saved else None)


Parquet engine not installed (pyarrow/fastparquet). Skipping Parquet save.


('bbc_with_subcategories_bertopic.csv', None)

In [ ]:
# Detailed view: top 10 subcategories per main Category with examples
from itertools import islice

def print_top_subcats_with_examples(frame: pd.DataFrame, per_category: int = 10, examples_per_subcat: int = 3):
    grouped = (
        frame.groupby(["Category", "Subcategory"])  # type: ignore
             .size()
             .reset_index(name="Count")
             .sort_values(["Category", "Count"], ascending=[True, False])
    )

    for category in grouped["Category"].unique():
        print(f"\n=== {category} ===")
        top_subcats = grouped[grouped["Category"] == category].head(per_category)
        for _, row in top_subcats.iterrows():
            subcat = row["Subcategory"]
            count = int(row["Count"])
            print(f"- {subcat}: {count}")
            examples = frame[(frame["Category"] == category) & (frame["Subcategory"] == subcat)]["Title"].head(examples_per_subcat).tolist()
            for ex in examples:
                print(f"    • {ex}")

print_top_subcats_with_examples(df, per_category=10, examples_per_subcat=3)



=== business ===
- Growth, Oil, Said: 464
    • Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited
    • Dollar gains on Greenspan speech The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise. And Alan Greenspan highlighted the US government's
    • Yukos unit buyer faces loan claim The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (£479m) loan. State-owned Rosneft bought the Yugansk unit for $9.3bn in
- Labour, Said, Would: 23
    • India calls for fair trade rules India, which attends the G7 meeting of seven leading industrialised nations on Friday, is unlikely to be cowed by its newcomer status. In London on Thursday ahead of t